<a href="https://colab.research.google.com/github/tomonari-masada/course2023-sml/blob/main/11_word_embeddings_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語の単語ベクトルをクラスタリング

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

## 1) spaCyの中規模日本語言語モデルのword2vec

* spaCyの中規模日本語モデルで、日本語の単語を300次元のベクトルで表現したデータを利用できる。
 * word2vecそのものについては、ここでは説明しません。
* 今回はこの単語ベクトルをクラスタリングして、意味の近い単語が同じクラスタに属しているかをチェックする。

In [ ]:
!python -m spacy download ja_core_news_md

In [ ]:
import spacy
nlp = spacy.load("ja_core_news_md")

* 単語ベクトルの次元を確認する。



In [ ]:
doc = nlp("これは何でしょうか。")
for token in doc:
  print(token, token.vector.shape)

* 文書全体のベクトル化もできる（今回は使わない）。

In [ ]:
doc = nlp("これは何でしょうか。")
doc.vector.shape

* 語彙サイズを確認する。

In [ ]:
len(nlp.vocab.strings)

* 単語ベクトルがゼロベクトルになっている単語があるようだ。

In [ ]:
nlp.vocab.get_vector("これ")

In [ ]:
nlp.vocab.has_vector("これ")

* ゼロベクトルでない単語ベクトルだけ集める。

In [ ]:
words = []
vectors = []
for word in tqdm(nlp.vocab.strings):
  if nlp.vocab.has_vector(word):
    vector = nlp.vocab.get_vector(word)
    if np.abs(vector).sum() > 0:
      words.append(word)
      vectors.append(vector)
words = np.array(words)
vectors = np.array(vectors)

In [ ]:
print(words.shape, vectors.shape)

* 試しに、最初の単語とその単語ベクトルを確認する



In [ ]:
print(words[0])
print(vectors[0])

* 「日本」という単語に最も近い10個の単語を表示させてみる。


In [ ]:
vec_jpn = nlp.vocab.get_vector("日本")
np.linalg.norm(vectors - vec_jpn, axis=1)
indices = np.argsort(np.linalg.norm(vectors - vec_jpn, axis=1))
print(words[indices[:11]])
print(np.linalg.norm(vectors - vec_jpn, axis=1)[indices[:11]])

* 'ハセベ', '日本', 'nippon'という３単語が同じベクトルになっているらしい。

* 「アメリカ」という単語については、どうか。

In [ ]:
vec_jpn = nlp.vocab.get_vector("アメリカ")
np.linalg.norm(vectors - vec_jpn, axis=1)
indices = np.argsort(np.linalg.norm(vectors - vec_jpn, axis=1))
print(words[indices[:11]])
print(np.linalg.norm(vectors - vec_jpn, axis=1)[indices[:11]])

* 'ガンジーブログ', 'America', '米西', 'メリカ', 'シアーズ', 'hanson', 'アメリカ', 'アメリカーナ', 'スムート', 'アメリカヘ'が全て同じベクトルになっているらしい。

## 2) 日本語BERTのword embeddings

* 東北大が提供している事前学習済みBERTのembeddings部分のみを利用する。
 * このBERTは日本語データで事前学習されている。

In [ ]:
!pip install transformers

In [ ]:
!pip install fugashi[unidic-lite]

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v3")

model = AutoModelForPreTraining.from_pretrained("cl-tohoku/bert-base-japanese-v3")

In [ ]:
list(tokenizer.vocab)[:10]

In [ ]:
len(tokenizer.vocab)

In [ ]:
words = np.array(list(tokenizer.vocab))

In [ ]:
model.bert

In [ ]:
vectors = model.bert.embeddings.word_embeddings.weight.data.numpy()

In [ ]:
vectors.shape

In [ ]:
words[0]

In [ ]:
vectors[0]

In [ ]:
tokenizer.convert_tokens_to_ids(['日本'])

In [ ]:
vec_jpn = vectors[tokenizer.convert_tokens_to_ids(['日本']),:]
np.linalg.norm(vectors - vec_jpn, axis=1)
indices = np.argsort(np.linalg.norm(vectors - vec_jpn, axis=1))
print(words[indices[:11]])
print(np.linalg.norm(vectors - vec_jpn, axis=1)[indices[:11]])

* クラスタ数の設定



In [ ]:
n_clusters = 100

* k-平均法によるクラスタリングを実行。

In [ ]:
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(vectors)

* クラスタリングの結果をcsvファイルとして保存。

In [ ]:
np.savetxt(f'bert_embed_cluster_centers_{n_clusters:d}.csv', kmeans.cluster_centers_, delimiter=',')
centers = kmeans.cluster_centers_

* k-平均法を実行するのではなく、クラスタの重心をファイルから読み込むときは、下のセルを実行。
 * パスは適当に書き換える。


In [ ]:
centers = np.loadtxt(f'bert_embed_cluster_centers_{n_clusters:d}.csv', delimiter=',')

In [ ]:
center = centers[0]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(words[indices[:20]])

### クラスタのサイズを調べる

* NumPyの配列に、いろいろな値が何回ずつ出てくるかを知るには、unique関数を使うと良い。

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)

In [ ]:
unique

In [ ]:
counts

* クラスタのインデックスをキーとし、そのサイズを値とする辞書を作る。

In [ ]:
size_dict = dict(zip(unique, counts))

* 辞書のエントリを、キーではなく値でソートする。

In [ ]:
sorted_clusters = [k for k, v in sorted(size_dict.items(), key=lambda item: item[1], reverse=True)]

In [ ]:
counts[sorted_clusters]

### サイズが最大のクラスタを調べる

In [ ]:
print(sorted_clusters)

In [ ]:
kmeans.labels_

In [ ]:
center = kmeans.cluster_centers_[sorted_clusters[0]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
' '.join(words[indices[:100]].tolist())

In [ ]:
filtered_indices = indices[kmeans.labels_[indices] == 56]
' '.join(words[filtered_indices[:100]].tolist())

### サイズが中間的なクラスタを調べる

In [ ]:
center = kmeans.cluster_centers_[sorted_clusters[49]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
' '.join(words[indices[:100]].tolist())

In [ ]:
kmeans.labels_[indices][:100]

In [ ]:
filtered_indices = indices[kmeans.labels_[indices] == sorted_clusters[49]]
' '.join(words[filtered_indices[:100]].tolist())

### 適当なクラスターを調べてみる

In [ ]:
center = kmeans.cluster_centers_[sorted_clusters[39]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
' '.join(words[indices[:100]].tolist())

In [ ]:
center = kmeans.cluster_centers_[sorted_clusters[29]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
' '.join(words[indices[:100]].tolist())